<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/AllEkphrasisvec4negpos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from google.colab import  drive
drive.mount('/content/drive')
import nltk
from nltk.corpus import stopwords
nltk.download('all')
device_name = tf.test.gpu_device_name()
print(device_name)
df_train = pd.read_csv('/content/drive/MyDrive/InfluenceAnalysis/Newdata/train_clean.csv',names=['Tweet','Emotion'])
df_test = pd.read_csv('/content/drive/MyDrive/InfluenceAnalysis/Newdata/test_clean.csv',names=['Tweet','Emotion'])
df_train = df_train.sample(frac=1)
df_test = df_train.sample(frac=1)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading pack

In [2]:
df_train.head(1000)

,Tweet,Emotion
5306,mikeabram link doe not work,0
752,thedailysurvey whi do we love those that we ca...,2
8880,pauline i know I am so excit we cannot go earl...,1
6834,freesat coverag sky dish is realli realli bad ...,2
4919,td_keepsitr haha awe well right now i could be...,1
...,...,...
9877,joeysmissmac blah got a shot at the hospit tak...,0
5965,hate live down here in fl miss ga like crazyli...,2
2492,sophiaf3f3 word I am alway up,0
1693,3wordsaftersex it hurt now,2


In [3]:
df_test.Emotion.unique()

array(['2', '0', '1', '3', 'emotion'], dtype=object)

In [4]:
df_train.nunique()

Tweet      12370
Emotion        5
dtype: int64

In [5]:
category_dict = {'0':[1,0,0,0],'1':[0,1,0,0],'2':[0,0,1,0],'3':[0,0,0,1],'emotion':[1,0,0,0]}
train_data_tweet = [x.lower().split() for x in df_train['Tweet']]
train_data_cat = np.array([category_dict[x] for x in df_train['Emotion']])
test_data_tweet = [x.lower().split() for x in df_test['Tweet']]
test_data_cat = np.array([category_dict[x] for x in df_test['Emotion']])

data_tweet = train_data_tweet + test_data_tweet

In [6]:
# Parameters
Min_count = 0
Embedding_size = 100
Window_size = 5
Negative_sampling = 00

In [7]:
w2v_sg = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=1)
w2v_sg.build_vocab(data_tweet)
w2v_sg.train(data_tweet, total_examples=w2v_sg.corpus_count, epochs=5)
w2v_cbow = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=0)
w2v_cbow.build_vocab(data_tweet)
w2v_cbow.train(data_tweet, total_examples=w2v_cbow.corpus_count, epochs=5)
w2v_sg.wv.init_sims(True)
w2v_cbow.wv.init_sims(True)


In [8]:
vocab_sg = w2v_sg.wv.vocab
vocab_sg = [x for x in vocab_sg]
vocab_cbow = w2v_cbow.wv.vocab
vocab_cbow = [x for x in vocab_cbow] 
vocab_glove = {}
with open("/content/drive/My Drive/InfluenceAnalysis/glove/glove.twitter.27B.100d.txt", 'r', encoding="utf-8") as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      vocab_glove[word] = vector

In [9]:
def save_dict_to_file(dic):
    f = open("/content/drive/My Drive/InfluenceAnalysis/vocab1.txt",'w')
    for i in dic:
      f.write(str(i)+"\t")
      f.write(str(w2v_sg.wv.get_vector(i)))
      f.write("\n")
    f.close()
save_dict_to_file(vocab_sg)

In [10]:
batch_size = 500
Max_input_size = max([len(x) for x in data_tweet])
mul_factor=1.5

In [11]:
positive = pd.read_csv('/content/drive/My Drive/InfluenceAnalysis/CodeMix/positive-words.csv')
positive = list(positive['0'][1:])
negative = pd.read_csv('/content/drive/My Drive/InfluenceAnalysis/CodeMix/negative-words.csv')
negative = list(negative['0'][2:])
pn_dict = {}
for i in range(len(positive)):
  pn_dict[positive[i]] = 1

for i in range(len(negative)):
  pn_dict[negative[i]] = -1

In [12]:
X_train = train_data_tweet
X_test = test_data_tweet
y_train = train_data_cat
y_test = test_data_cat 
X_train = [nltk.pos_tag(x) for x in X_train]
X_test = [nltk.pos_tag(x) for x in X_test]
X_train = [[(i[0],i[1],pn_dict.get(i[0],1)) for i in x] for x in X_train]
X_test = [[(i[0],i[1],pn_dict.get(i[0],1)) for i in x] for x in X_test]
result_table = [0,0,0,0,0,0,0,0,0]

In [13]:
def vec_gen(w2v,vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(w2v.wv.word_vec(i[0]),i[1],i[2]) for i in x if i[0] in vocab]
        l=len(tweet)
        for i in range(l):
          tweet[i]=((tweet[i][0]*tweet[i][2]),tweet[i][1],tweet[i][2])
        for i in range(l):
          if tweet[i][1][:2]=='JJ':
            for z in range(max(i-2,0),min(i+3,l)):
              if tweet[z][1][:2]=='NN' or tweet[z][1][:2]=='VB':
                tweet[z]=((tweet[z][0]+tweet[i][0])*mul_factor,tweet[z][1])
          if tweet[i][1][:2]=='RB':
            for z in range(max(i-2,0),min(i+3,l)):
              if tweet[z][1][:2]=='VB':
                tweet[z]=((tweet[z][0]+tweet[i][0])*mul_factor,tweet[z][1])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)          
      
def glove_gen(vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(vocab[i[0]],i[1],i[2]) for i in x if i[0] in vocab.keys()]
        l=len(tweet)
        for i in range(l):
          tweet[i]=((tweet[i][0]*tweet[i][2]),tweet[i][1],tweet[i][2])
        for i in range(l):
          if tweet[i][1][:2]=='JJ':
            for z in range(max(i-2,0),min(i+3,l)):
              if tweet[z][1][:2]=='NN' or tweet[z][1][:2]=='VB':
                tweet[z]=((tweet[z][0]+tweet[i][0])*mul_factor,tweet[z][1])
          if tweet[i][1][:2]=='RB':
            for z in range(max(i-2,0),min(i+3,l)):
              if tweet[z][1][:2]=='VB':
                tweet[z]=((tweet[z][0]+tweet[i][0])*mul_factor,tweet[z][1])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)

In [14]:
epochs = 40

In [15]:
model_sg = tf.keras.Sequential()
model_sg.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg.add(tf.keras.layers.LSTM(50))
model_sg.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[0]=history[1]
print("SG_LSTM")


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/40
24/24 [==============================] - 6s 243ms/step - loss: 1.2620 - accuracy: 0.5692
Epoch 2/40
24/24 [==============================] - 6s 251ms/step - loss: 1.0067 - accuracy: 0.5869
Epoch 3/40
24/24 [==============================] - 6s 250ms/step - loss: 0.9836 - accuracy: 0.5869
Epoch 4/40
24/24 [==============================] - 6s 253ms/step - loss: 0.9824 - accuracy: 0.5869
Epoch 5/40
24/24 [==============================] - 6s 252ms/step - loss: 0.9803 - accuracy: 0.5869
Epoch 6/40
24/24 [==============================] - 6s 249ms/step - loss: 0.9765 - accuracy: 0.5869
Epoch 7/40
24/24 [==============================] - 6s 249ms/step - loss: 0.9725 - accuracy: 0.5869
Epoch 8/40
24/24 [==============================] - 6s 252ms/step - loss: 0.9678 - accuracy: 0.5866
Epoch 9/40
24/24 [==============================] - 6s 248ms/step - loss: 0.9649 - accuracy: 0.5866
Epoch 10/40
24/24 [=====

In [16]:
model_sg_bi = tf.keras.Sequential()
model_sg_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_sg_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg_bi.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg_bi.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[1]=history[1]
print("SG_BI")

Epoch 1/40
24/24 [==============================] - 6s 249ms/step - loss: 1.2075 - accuracy: 0.5446
Epoch 2/40
24/24 [==============================] - 6s 253ms/step - loss: 0.9941 - accuracy: 0.5869
Epoch 3/40
24/24 [==============================] - 6s 254ms/step - loss: 0.9740 - accuracy: 0.5870
Epoch 4/40
24/24 [==============================] - 6s 256ms/step - loss: 0.9681 - accuracy: 0.5870
Epoch 5/40
24/24 [==============================] - 6s 253ms/step - loss: 0.9611 - accuracy: 0.5872
Epoch 6/40
24/24 [==============================] - 6s 250ms/step - loss: 0.9549 - accuracy: 0.5869
Epoch 7/40
24/24 [==============================] - 6s 256ms/step - loss: 0.9493 - accuracy: 0.5878
Epoch 8/40
24/24 [==============================] - 6s 254ms/step - loss: 0.9444 - accuracy: 0.5890
Epoch 9/40
24/24 [==============================] - 6s 250ms/step - loss: 0.9404 - accuracy: 0.5895
Epoch 10/40
24/24 [==============================] - 6s 252ms/step - loss: 0.9362 - accuracy: 0.5913

In [17]:
model_sg_rnn = tf.keras.Sequential()
model_sg_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_rnn.add(tf.keras.layers.SimpleRNN(50))
model_sg_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg_rnn.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg_rnn.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[2]=history[1]
print("SG_RNN")

Epoch 1/40
24/24 [==============================] - 6s 260ms/step - loss: 1.1100 - accuracy: 0.5458
Epoch 2/40
24/24 [==============================] - 6s 267ms/step - loss: 0.9925 - accuracy: 0.5863
Epoch 3/40
24/24 [==============================] - 6s 268ms/step - loss: 0.9852 - accuracy: 0.5868
Epoch 4/40
24/24 [==============================] - 7s 273ms/step - loss: 0.9844 - accuracy: 0.5869
Epoch 5/40
24/24 [==============================] - 6s 264ms/step - loss: 0.9838 - accuracy: 0.5871
Epoch 6/40
24/24 [==============================] - 6s 263ms/step - loss: 0.9831 - accuracy: 0.5872
Epoch 7/40
24/24 [==============================] - 6s 266ms/step - loss: 0.9820 - accuracy: 0.5872
Epoch 8/40
24/24 [==============================] - 6s 260ms/step - loss: 0.9799 - accuracy: 0.5874
Epoch 9/40
24/24 [==============================] - 6s 266ms/step - loss: 0.9758 - accuracy: 0.5879
Epoch 10/40
24/24 [==============================] - 6s 262ms/step - loss: 0.9696 - accuracy: 0.5880

In [18]:
model_cbow = tf.keras.Sequential()
model_cbow.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow.add(tf.keras.layers.LSTM(50))
model_cbow.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[3]=history[1]
print("CBOW_LSTM")

Epoch 1/40
24/24 [==============================] - 6s 251ms/step - loss: 1.2425 - accuracy: 0.5744
Epoch 2/40
24/24 [==============================] - 6s 252ms/step - loss: 1.0018 - accuracy: 0.5869
Epoch 3/40
24/24 [==============================] - 6s 251ms/step - loss: 0.9854 - accuracy: 0.5869
Epoch 4/40
24/24 [==============================] - 6s 255ms/step - loss: 0.9841 - accuracy: 0.5869
Epoch 5/40
24/24 [==============================] - 6s 251ms/step - loss: 0.9830 - accuracy: 0.5869
Epoch 6/40
24/24 [==============================] - 6s 250ms/step - loss: 0.9810 - accuracy: 0.5869
Epoch 7/40
24/24 [==============================] - 6s 250ms/step - loss: 0.9770 - accuracy: 0.5869
Epoch 8/40
24/24 [==============================] - 6s 249ms/step - loss: 0.9718 - accuracy: 0.5869
Epoch 9/40
24/24 [==============================] - 6s 248ms/step - loss: 0.9659 - accuracy: 0.5869
Epoch 10/40
24/24 [==============================] - 6s 265ms/step - loss: 0.9602 - accuracy: 0.5867

In [19]:
model_cbow_bi = tf.keras.Sequential()
model_cbow_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_cbow_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow_bi.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow_bi.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[4]=history[1]
print("CBOW_BI")

Epoch 1/40
24/24 [==============================] - 6s 252ms/step - loss: 1.2170 - accuracy: 0.5454
Epoch 2/40
24/24 [==============================] - 6s 257ms/step - loss: 0.9915 - accuracy: 0.5870
Epoch 3/40
24/24 [==============================] - 6s 257ms/step - loss: 0.9748 - accuracy: 0.5869
Epoch 4/40
24/24 [==============================] - 6s 257ms/step - loss: 0.9698 - accuracy: 0.5870
Epoch 5/40
24/24 [==============================] - 6s 258ms/step - loss: 0.9647 - accuracy: 0.5870
Epoch 6/40
24/24 [==============================] - 6s 256ms/step - loss: 0.9586 - accuracy: 0.5872
Epoch 7/40
24/24 [==============================] - 6s 254ms/step - loss: 0.9525 - accuracy: 0.5876
Epoch 8/40
24/24 [==============================] - 6s 261ms/step - loss: 0.9478 - accuracy: 0.5879
Epoch 9/40
24/24 [==============================] - 6s 258ms/step - loss: 0.9434 - accuracy: 0.5888
Epoch 10/40
24/24 [==============================] - 6s 256ms/step - loss: 0.9392 - accuracy: 0.5890

In [20]:
model_cbow_rnn = tf.keras.Sequential()
model_cbow_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_rnn.add(tf.keras.layers.SimpleRNN(50))
model_cbow_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow_rnn.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow_rnn.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[5]=history[1]
print("CBOW_RNN")

Epoch 1/40
24/24 [==============================] - 6s 266ms/step - loss: 1.1167 - accuracy: 0.5263
Epoch 2/40
24/24 [==============================] - 6s 264ms/step - loss: 0.9869 - accuracy: 0.5868
Epoch 3/40
24/24 [==============================] - 6s 267ms/step - loss: 0.9856 - accuracy: 0.5869
Epoch 4/40
24/24 [==============================] - 7s 288ms/step - loss: 0.9856 - accuracy: 0.5869
Epoch 5/40
24/24 [==============================] - 7s 281ms/step - loss: 0.9855 - accuracy: 0.5869
Epoch 6/40
24/24 [==============================] - 6s 267ms/step - loss: 0.9853 - accuracy: 0.5869
Epoch 7/40
24/24 [==============================] - 6s 264ms/step - loss: 0.9853 - accuracy: 0.5869
Epoch 8/40
24/24 [==============================] - 7s 272ms/step - loss: 0.9853 - accuracy: 0.5869
Epoch 9/40
24/24 [==============================] - 6s 267ms/step - loss: 0.9853 - accuracy: 0.5869
Epoch 10/40
24/24 [==============================] - 6s 269ms/step - loss: 0.9852 - accuracy: 0.5869

In [21]:
epochs=40
Embedding_size=100

In [22]:
model_glove = tf.keras.Sequential()
model_glove.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove.add(tf.keras.layers.LSTM(50))
model_glove.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[6]=history[1]
print("GLOVE_LSTM")


Epoch 1/40
24/24 [==============================] - 2s 66ms/step - loss: 1.2234 - accuracy: 0.5639
Epoch 2/40
24/24 [==============================] - 2s 67ms/step - loss: 0.9971 - accuracy: 0.5869
Epoch 3/40
24/24 [==============================] - 2s 68ms/step - loss: 0.9784 - accuracy: 0.5869
Epoch 4/40
24/24 [==============================] - 2s 68ms/step - loss: 0.9731 - accuracy: 0.5869
Epoch 5/40
24/24 [==============================] - 2s 68ms/step - loss: 0.9710 - accuracy: 0.5869
Epoch 6/40
24/24 [==============================] - 2s 66ms/step - loss: 0.9676 - accuracy: 0.5870
Epoch 7/40
24/24 [==============================] - 2s 67ms/step - loss: 0.9623 - accuracy: 0.5870
Epoch 8/40
24/24 [==============================] - 2s 67ms/step - loss: 0.9581 - accuracy: 0.5888
Epoch 9/40
24/24 [==============================] - 2s 67ms/step - loss: 0.9514 - accuracy: 0.5924
Epoch 10/40
24/24 [==============================] - 2s 65ms/step - loss: 0.9438 - accuracy: 0.5947
Epoch 11/

In [23]:
model_glove_bi = tf.keras.Sequential()
model_glove_bi.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_glove_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove_bi.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove_bi.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[7]=history[1]
print("GLOVE_BI")

Epoch 1/40
24/24 [==============================] - 2s 72ms/step - loss: 1.1712 - accuracy: 0.4874
Epoch 2/40
24/24 [==============================] - 2s 75ms/step - loss: 0.9701 - accuracy: 0.5870
Epoch 3/40
24/24 [==============================] - 2s 74ms/step - loss: 0.9488 - accuracy: 0.5872
Epoch 4/40
24/24 [==============================] - 2s 77ms/step - loss: 0.9332 - accuracy: 0.5882
Epoch 5/40
24/24 [==============================] - 2s 76ms/step - loss: 0.9179 - accuracy: 0.5923
Epoch 6/40
24/24 [==============================] - 2s 76ms/step - loss: 0.9065 - accuracy: 0.5975
Epoch 7/40
24/24 [==============================] - 2s 74ms/step - loss: 0.8965 - accuracy: 0.6029
Epoch 8/40
24/24 [==============================] - 2s 75ms/step - loss: 0.8868 - accuracy: 0.6092
Epoch 9/40
24/24 [==============================] - 2s 75ms/step - loss: 0.8771 - accuracy: 0.6151
Epoch 10/40
24/24 [==============================] - 2s 75ms/step - loss: 0.8676 - accuracy: 0.6187
Epoch 11/

In [24]:
model_glove_rnn = tf.keras.Sequential()
model_glove_rnn.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove_rnn.add(tf.keras.layers.SimpleRNN(50))
model_glove_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove_rnn.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove_rnn.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[8]=history[1]
print("GLOVE_RNN")

Epoch 1/40
24/24 [==============================] - 2s 87ms/step - loss: 1.1294 - accuracy: 0.5277
Epoch 2/40
24/24 [==============================] - 2s 86ms/step - loss: 0.9892 - accuracy: 0.5862
Epoch 3/40
24/24 [==============================] - 2s 89ms/step - loss: 0.9829 - accuracy: 0.5867
Epoch 4/40
24/24 [==============================] - 2s 88ms/step - loss: 0.9803 - accuracy: 0.5866
Epoch 5/40
24/24 [==============================] - 2s 87ms/step - loss: 0.9776 - accuracy: 0.5867
Epoch 6/40
24/24 [==============================] - 2s 87ms/step - loss: 0.9774 - accuracy: 0.5872
Epoch 7/40
24/24 [==============================] - 2s 88ms/step - loss: 0.9736 - accuracy: 0.5868
Epoch 8/40
24/24 [==============================] - 2s 86ms/step - loss: 0.9685 - accuracy: 0.5880
Epoch 9/40
24/24 [==============================] - 2s 88ms/step - loss: 0.9661 - accuracy: 0.5889
Epoch 10/40
24/24 [==============================] - 2s 87ms/step - loss: 0.9618 - accuracy: 0.5911
Epoch 11/

In [25]:
model_sg1 = tf.keras.Sequential()
model_sg1.add(tf.keras.Input(shape=(Max_input_size,100)))
model_sg1.add(tf.keras.layers.LSTM(64))
model_sg1.add(tf.keras.layers.Dropout(0.2))
model_sg1.add(tf.keras.layers.Dense(64,activation='relu'))
model_sg1.add(tf.keras.layers.Dropout(0.2))
model_sg1.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg1.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg1.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg1.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("SG_LSTM")

Epoch 1/40
24/24 [==============================] - 6s 244ms/step - loss: 1.2387 - accuracy: 0.5730
Epoch 2/40
24/24 [==============================] - 6s 245ms/step - loss: 1.0254 - accuracy: 0.5867
Epoch 3/40
24/24 [==============================] - 6s 247ms/step - loss: 1.0017 - accuracy: 0.5857
Epoch 4/40
24/24 [==============================] - 6s 244ms/step - loss: 0.9954 - accuracy: 0.5869
Epoch 5/40
24/24 [==============================] - 6s 246ms/step - loss: 0.9904 - accuracy: 0.5865
Epoch 6/40
24/24 [==============================] - 6s 246ms/step - loss: 0.9857 - accuracy: 0.5867
Epoch 7/40
24/24 [==============================] - 6s 253ms/step - loss: 0.9800 - accuracy: 0.5868
Epoch 8/40
24/24 [==============================] - 6s 247ms/step - loss: 0.9761 - accuracy: 0.5871
Epoch 9/40
24/24 [==============================] - 6s 244ms/step - loss: 0.9745 - accuracy: 0.5870
Epoch 10/40
24/24 [==============================] - 6s 247ms/step - loss: 0.9703 - accuracy: 0.5869

In [26]:
print("\tLstm\t\t\tBiLstm\t\t\tRnn")
print("sg",result_table[:3])
print("cbow",result_table[3:6])
print("glove",result_table[6:9])

	Lstm			BiLstm			Rnn
sg [0.7015833258628845, 0.6775000095367432, 0.6730833053588867]
cbow [0.6918333172798157, 0.6694999933242798, 0.5979166626930237]
glove [0.7476666569709778, 0.8194166421890259, 0.6447499990463257]


In [27]:
print(result_table[9])

0.7009166479110718
